In [52]:
!pip install load_dotenv


In [53]:
import openai
import re
import httpx
import os
from dotenv import load_dotenv

_ = load_dotenv()
from openai import OpenAI

In [54]:
import os
from openai import OpenAI
from google.colab import userdata

# Fetch secret and set it as an environment variable
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

# Now initialize the client
client = OpenAI()


In [55]:
chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [57]:
import re
from openai import OpenAI
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")


client = OpenAI()

class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o",
            temperature=0.5,
            messages=self.messages
        )
        return completion.choices[0].message.content


In [58]:
content_strategist_prompt = """
You are a technical content strategist helping software developers stay sharp.

Ask the user a few quick questions to understand their:
- Interests (e.g., backend, AI, DevOps)
- Skill level (beginner, intermediate, advanced)
- Purpose (learn, teach, write, share, explore)
- Preferred languages/tools (optional)

Then suggest 3-5 specific, trending, or high-impact tech topics they could write or learn about.
""".strip()

knowledge_researcher_prompt = """
You are a technical researcher helping developers understand complex topics.

Given a topic, summarize the most important concepts, common use-cases, and current best practices.
Also include:
- Key terminology
- Real-world applications
- Common pitfalls
- A simple code example if applicable
""".strip()

technical_writer_prompt = """
You are a skilled technical writer creating blog posts for software engineers.

Generate a blog post with:
- Introduction
- 2–3 detailed sections
- Code snippets with comments
- A practical conclusion

Use Markdown formatting with code blocks.
Write for clarity, depth, and engagement.
""".strip()

editor_optimizer_prompt = """
You are a technical editor. Your tasks:
- Fix grammar and improve flow
- Enhance technical clarity
- Add Markdown formatting
- Maintain developer tone

Make the content publication-ready without losing its technical depth.
""".strip()

quiz_generator_prompt = """
You are a quiz generator for technical readers.

Given an article, create a quiz with:
- 3 multiple-choice questions (MCQs)
- Each question with 4 options
- One correct answer per question

Example format:
Q1. ...
A. ...
B. ...
C. ...
D. ...
Answer: B
""".strip()


In [59]:
def suggest_topics(context):
    agent = Agent(content_strategist_prompt)
    return agent(context)

def research_topic(topic):
    agent = Agent(knowledge_researcher_prompt)
    return agent(topic)

def write_article(research_summary):
    agent = Agent(technical_writer_prompt)
    return agent(research_summary)

def optimize_article(article_draft):
    agent = Agent(editor_optimizer_prompt)
    return agent(article_draft)

def generate_quiz(article_text):
    agent = Agent(quiz_generator_prompt)
    return agent(article_text)


In [60]:
def run_full_pipeline(user_context):
    print("💡 Suggesting topics...")
    topics = suggest_topics(user_context)
    print(topics)

    topic = input("\n✍️ Choose a topic to continue with: ").strip()

    print("\n🔍 Researching topic...")
    research = research_topic(topic)
    print(research)

    print("\n📝 Writing article...")
    article = write_article(research)
    print(article)

    print("\n🧹 Optimizing article...")
    polished_article = optimize_article(article)
    print(polished_article)

    print("\n🧪 Generating quiz...")
    quiz = generate_quiz(polished_article)
    print(quiz)

    return {
        "topic": topic,
        "research": research,
        "article": article,
        "polished_article": polished_article,
        "quiz": quiz
    }


In [61]:
output = run_full_pipeline("I’m a backend developer, intermediate level. I want to write to solidify my understanding and help others.")


💡 Suggesting topics...
Great! As an intermediate backend developer looking to write and solidify your understanding while helping others, here are a few trending and high-impact topics you could explore:

1. **Microservices Architecture**: Dive into the principles and practices of designing and implementing microservices. You could explore topics like service discovery, inter-service communication, and managing data consistency across services.

2. **GraphQL vs. REST**: Compare and contrast GraphQL and REST APIs. Discuss their use cases, benefits, and drawbacks, and provide examples of when to use each approach in backend development.

3. **Serverless Computing**: Explore the serverless paradigm and its impact on backend development. Discuss how to design and deploy serverless applications using platforms like AWS Lambda or Azure Functions.

4. **Containerization with Docker and Kubernetes**: Write about how Docker and Kubernetes are used in backend development to manage containerized 

In [28]:
!pip install gradio --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00


In [29]:
import gradio as gr

def full_agent_pipeline(user_input):
    try:
        # Suggest Topics
        topics = suggest_topics(user_input)

        # Extract first topic (for now)
        first_topic = re.findall(r'\d+\.\s*(.*)', topics)
        selected_topic = first_topic[0] if first_topic else user_input

        # Research
        research = research_topic(selected_topic)

        # Write
        article = write_article(research)

        # Optimize
        polished = optimize_article(article)

        # Quiz
        quiz = generate_quiz(polished)

        # Return all sections
        return (
            topics,
            selected_topic,
            research,
            polished,
            quiz
        )

    except Exception as e:
        return ("Error", "Error", str(e), "", "")

# Define Gradio app
app = gr.Interface(
    fn=full_agent_pipeline,
    inputs=gr.Textbox(label="🧠 Describe your background, goals, or interest area"),
    outputs=[
        gr.Textbox(label="🎯 Suggested Topics"),
        gr.Textbox(label="✅ Selected Topic"),
        gr.Textbox(label="📚 Research Summary"),
        gr.Textbox(label="✍️ Final Article"),
        gr.Textbox(label="🧪 Quiz for Readers"),
    ],
    title="🛠️ Daily Tech Knowledge Booster",
    description="This AI agent helps you discover, learn, write, and quiz yourself on developer topics. Just tell it what you're into!",
    theme="soft"
)


In [30]:
app.launch(inline=True)


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c873286dd09fc38d39.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [33]:
import requests
Sarvam_API_Key=userdata.get("SARVAM_API")
def chunk_text(text, max_length=1000):
    """Splits text into chunks of at most max_length characters while preserving word boundaries."""
    chunks = []

    while len(text) > max_length:
        split_index = text.rfind(" ", 0, max_length)  # Find the last space within limit
        if split_index == -1:
            split_index = max_length  # No space found, force split at max_length

        chunks.append(text[:split_index].strip())  # Trim spaces before adding
        text = text[split_index:].lstrip()  # Remove leading spaces for the next chunk

    if text:
        chunks.append(text.strip())  # Add the last chunk

    return chunks
def translate_text_sarvam(text, source_lang="en-IN", target_lang="hi-IN", api_key=None):
    url = "https://api.sarvam.ai/transliterate"
    headers = {
        "api-subscription-key": api_key,
        "Content-Type": "application/json"
    }

    chunks = chunk_text(text, max_length=1000)
    translated_texts = []

    for idx, chunk in enumerate(chunks):
        payload = {
            "input": chunk,
            "source_language_code": source_lang,
            "target_language_code": target_lang,
            "spoken_form": True,
            "numerals_format": "international"
        }

        response = requests.post(url, json=payload, headers=headers)
        print(response)

        if response.status_code == 200:
            translated = response.json().get("transliterated_text", "")
            translated_texts.append(translated)
        else:
            translated_texts.append(f"[Error {response.status_code}]")

    return "\n".join(translated_texts)


In [34]:
import base64
import wave
import os

def generate_audio_sarvam(text, target_lang="hi-IN", speaker="neel", model="bulbul:v1", api_key=None, output_prefix="output"):
    url = "https://api.sarvam.ai/text-to-speech"
    headers = {
        "api-subscription-key": api_key,
        "Content-Type": "application/json"
    }

    chunk_size = 500
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    audio_paths = []

    for i, chunk in enumerate(chunks):
        payload = {
            "inputs": [chunk],
            "target_language_code": target_lang,
            "speaker": speaker,
            "model": model,
            "pitch": 0,
            "pace": 1.0,
            "loudness": 1.0,
            "enable_preprocessing": True
        }

        response = requests.post(url, json=payload, headers=headers)
        if response.status_code == 200:
            audio_base64 = response.json()["audios"][0]
            audio_binary = base64.b64decode(audio_base64)

            filename = f"{output_prefix}{i}.wav"
            with wave.open(filename, "wb") as wav_file:
                wav_file.setnchannels(1)
                wav_file.setsampwidth(2)
                wav_file.setframerate(22050)
                wav_file.writeframes(audio_binary)

            audio_paths.append(filename)
        else:
            print(f"[Error {i}]: {response.status_code}, {response.text}")

    return audio_paths


In [35]:
# Sample usage after pipeline output
translated_article = translate_text_sarvam(
    text=output["polished_article"],
    source_lang="en-IN",
    target_lang="hi-IN", # You can switch this to other supported lang codes
    api_key=Sarvam_API_Key
)
print(translated_article)


<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
हैश बिल्डिंग अ रेस्टफुल ए पी आई विथ फ्लैस्क: अ स्टेप बाय स्टेप गाइड रेस्ट ए पी आईज़ आर एसेंशियल इन वेब डेवलपमेंट, प्रोवाइडिंग अ स्ट्रेटफॉरवर्ड येट पावरफुल मीन्स फॉर क्लाइंट सर्वर कम्युनिकेशन। इन दिस गाइड, वी एपोस्ट्रोफी डेल्व इनटू बिल्डिंग अ रेस्टफुल ए पी आई यूज़िंग फ़्लास्क, अ लाइटवेट एंड फ़्लेक्सिबल पायथन वेब फ्रेमवर्क। वील एक्सप्लोर की कॉन्सेप्ट्स, इम्प्लीमेंटेशन डिटेल्स, एंड बेस्ट प्रैक्टिसेस टू एन्शुर योर ए पी आई इज़ रोबस्ट एंड यूज़र फ्रेंडली। हैश इंट्रोडक्शन टू रेस्ट एंड फ्लास्क रेस्ट (रिप्रेजेंटेशनल स्टेट ट्रांसफर) इज़ एन आर्किटेक्चरल स्टाइल दैट यूटिलाइज़स स्टैंडर्ड एच टी टी पी मेथड्स टू परफॉर्म ऑपरेशन्स ऑन रिसोर्सेज़। इट एम्फसाइजेस स्टैलेज़नेस एंड अ यूनिफ़ॉर्म इंटरफ़ेस, मेकिंग इट आइडियल फ़ॉर स्केलेबल वेब सर्विसेज। फ्लैस्क इज़ अ माइक्रो वेब फ्रेमवर्क फॉर पायथन, नोन फॉर इट्स सिम्प्लिसिटी एंड ईज़ ऑफ़ यूज़, मेकिंग इट एन एक्सीलेंट चॉइस फॉर डेवलपिंग रेस्टफुल ए पी आईज़। हैश सेटिंग अप योर फ्लास्क एनवायरनमेंट बिफोर डाइव

In [36]:
audio_files = generate_audio_sarvam(
    text=translated_article,
    target_lang="hi-IN",
    speaker="neel",  # Change to other voices if needed
    model="bulbul:v1",
    api_key=Sarvam_API_Key,
    output_prefix="translated_article"
)


In [37]:

preferred_lang = "hi-IN"


preferred_speaker = "neel"

In [38]:
def run_full_pipeline(user_context, language="hi-IN", speaker="neel", api_key=None):
    print("💡 Suggesting topics...")
    topics = suggest_topics(user_context)
    print(topics)

    topic = input("\n✍️ Choose a topic to continue with: ").strip()

    print(f"\n🔍 Researching topic: {topic}")
    research = research_topic(topic)
    print("\n📚 Research Summary:\n", research)

    print("\n📝 Writing article...")
    article = write_article(research)
    print("\n🧾 Article Draft:\n", article)

    print("\n🧹 Optimizing article...")
    polished_article = optimize_article(article)
    print("\n✅ Final Polished Article:\n", polished_article)

    print("\n🧪 Generating quiz...")
    quiz = generate_quiz(polished_article)
    print("\n🧠 Quiz:\n", quiz)

    # Optional Translation
    print(f"\n🌐 Translating article to {language}...")
    translated = translate_text_sarvam(polished_article, target_lang=language, api_key=api_key)
    print("\n🈯 Translated Article:\n", translated)

    # Optional TTS
    print(f"\n🔊 Generating TTS audio in {language} using voice '{speaker}'...")
    audio_paths = generate_audio_sarvam(translated, target_lang=language, speaker=speaker, api_key=api_key)

    return {
        "topic": topic,
        "research": research,
        "article": article,
        "polished_article": polished_article,
        "quiz": quiz,
        "translated_article": translated,
        "audio_files": audio_paths
    }


In [39]:
# Set your Sarvam API Key first
SARVAM_API_KEY=userdata.get("SARVAM_API")


output = run_full_pipeline(
    user_context="I'm an intermediate backend developer. I want to write to deepen my knowledge.",
    language="kn-IN",         # Kannada
    speaker="neel",
    api_key=SARVAM_API_KEY
)


💡 Suggesting topics...
Great! To tailor some suggestions for you, could you let me know if there are any specific programming languages or tools you're particularly interested in or currently using? Additionally, are there any particular areas within backend development that intrigue you, such as databases, APIs, or cloud services?

✍️ Choose a topic to continue with: service now catalogue

🔍 Researching topic: service now catalogue

📚 Research Summary:
 ServiceNow Catalog is a feature within the ServiceNow platform that enables organizations to create and manage a centralized service catalog. This catalog provides a user-friendly interface for employees to request and access various IT and business services. It streamlines the process of service delivery by automating workflows, approvals, and fulfillment processes.

### Key Concepts

- **Service Catalog**: A collection of services that an organization offers to its users, typically categorized for easy navigation.
- **Request Item**:

In [48]:
import gradio as gr
import re

def gradio_run_pipeline(user_context, language, speaker, api_key):
    try:
        print("💡 Suggesting topics...")
        topics = suggest_topics(user_context)

        # Check if it's a follow-up prompt rather than actual topic suggestions
        if isinstance(topics, str) and "Interests" in topics and "Skill Level" in topics:
            return (
                "Please answer the following to get better topic suggestions:\n\n" + topics,
                "", "", "", "", "", ""
            )

        # Extract first topic (from numbered list or list object)
        parsed_topics = re.findall(r'\d+\.\s*(.*)', topics) if isinstance(topics, str) else topics
        selected_topic = parsed_topics[0] if parsed_topics else user_context

        print(f"\n🔍 Researching topic: {selected_topic}")
        research = research_topic(selected_topic)

        print("\n📝 Writing article...")
        article = write_article(research)

        print("\n🧹 Optimizing article...")
        polished_article = optimize_article(article)

        print("\n🧪 Generating quiz...")
        quiz = generate_quiz(polished_article)

        print(f"\n🌐 Translating article to {language}...")
        translated = translate_text_sarvam(polished_article, target_lang=language, api_key=api_key)

        print(f"\n🔊 Generating TTS audio in {language} using voice '{speaker}'...")
        audio_paths = generate_audio_sarvam(translated, target_lang=language, speaker=speaker, api_key=api_key)

        return (
            topics,
            selected_topic,
            research,
            polished_article,
            quiz,
            translated,
            audio_paths[0] if audio_paths else "No audio generated"
        )

    except Exception as e:
        return "Error", "Error", str(e), "", "", "", ""

# Gradio Interface
demo = gr.Interface(
    fn=gradio_run_pipeline,
    inputs=[
        gr.Textbox(label="🧠 Your Background or Interest", placeholder="e.g. I'm curious about blockchain apps for healthcare"),
        gr.Textbox(label="🌐 Language Code", value="hi-IN"),
        gr.Textbox(label="🔊 Speaker", value="neel"),
        gr.Textbox(label="🔐 API Key", type="password", placeholder="Enter your API key"),
    ],
    outputs=[
        gr.Textbox(label="🧩 Suggested Topics or Follow-up Questions"),
        gr.Textbox(label="✅ Selected Topic"),
        gr.Textbox(label="📚 Research Summary"),
        gr.Textbox(label="✍️ Final Article"),
        gr.Textbox(label="🧪 Quiz for Readers"),
        gr.Textbox(label="🌐 Translated Article"),
        gr.Audio(label="🔊 TTS Audio"),
    ],
    title="🤖 Smart Knowledge Generator",
    description="Start by entering your interest or background. If needed, the AI will ask follow-up questions to help suggest better topics.",
    theme="soft"
)

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://129d0a820c1e60357b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
